In [1]:
import numpy as np
import pickle
import neuraltda.SimplicialComplex as sc
import matplotlib.pyplot as plt
import scipy.linalg as spla
%matplotlib inline
reload(sc)

<module 'neuraltda.SimplicialComplex' from '/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/SimplicialComplex.pyc'>

In [ ]:
resF = ['/Volumes/gentner/btheilma/experiments/B1075/phy041216/Pen01_Lft_AP300_ML700__Site03_Z2700__B1075_cat_P01_S03_1/20170127T183908Z-10.0-5.0-4.0-TotalTopology.pkl',
        '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site01_Z3000__B1235_cat_P02_S01_1/20170127T235347Z-10.0-5.0-4.0-TotalTopology.pkl',
        '/Volumes/gentner/btheilma/experiments/B1083/klusta/phy040516/Pen03_Lft_AP0_ML1000__Site03_Z2700__B1083_cat_P03_S03_1/20170127T202857Z-10.0-5.0-4.0-TotalTopology.pkl']

resF = ['/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site01_Z3000__B1235_cat_P02_S01_1/20170127T235347Z-10.0-5.0-4.0-TotalTopology.pkl',
        '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/20170128T005306Z-10.0-5.0-4.0-TotalTopology.pkl',
        '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site04_Z3800__B1235_cat_P02_S04_1/20170128T005334Z-10.0-5.0-4.0-TotalTopology.pkl']
for resfname in resF:
    with open(resfname, 'r') as fd:
        resDict = pickle.load(fd)
        print(resDict)
        ec = 0
        for ind, num in enumerate(resDict['raw']['bettis'][1][1:]):
            ec = ec + (-1)**ind * num
        print(ec)
        

In [18]:
def union(a, b):
    return list(set(a) | set(b))

def simplexUnion(E1, E2):
    ''' Calculate the union of two simplicial complexes represented as lists of generators
    '''
    sortedE = sorted([E1, E2], key=len)
    maxlen = len(sortedE[1])
    minlen = len(sortedE[0])
    for i in range(maxlen-minlen):
        sortedE[0].append([])
    newE = []
    for ind in range(maxlen):
        newE.append(sorted(union(sortedE[0][ind], sortedE[1][ind])))
    return newE
    
def primaryFaces(Q):
    L = []
    d = len(Q)
    Q = list(Q)
    Q.extend(Q[:d-2])
    for ind in range(d):
        s = (Q[ind:ind+(d-1)])
        L.append(tuple(sorted(s)))
    return L

def simplicialChainGroups(maxsimps):
    maxdim = max([len(s) for s in maxsimps])
    E=[[] for ind in range(maxdim+2)]
    K = list(maxsimps)
    while(len(K) > 0):
        Q = K.pop(0)
        L = primaryFaces(Q)
        k = len(Q)-1
        K = union(K, L)
        E[k] = union(E[k], L)
        E[k+1] = union(E[k+1], {Q})
    for k in range(len(E)):
        E[k] = sorted(E[k])
    return E

def boundaryOperator(Q):
    sgn = 1
    c = dict()
    Ql = list(Q)
    for ind in range(len(Ql)):
        n = Ql.pop(ind)
        c[tuple(Ql)] = sgn
        Ql.insert(ind, n)
        sgn = -1*sgn
    return c

def canonicalCoordinates(c, K):
    v = np.zeros(len(K))
    for ind in range(len(K)):
        if c.has_key(K[ind]):
            v[ind] = c[K[ind]]
    return v

def boundaryOperatorMatrix(E):
    
    nmat = len(E)-1
    D = [[] for i in range(nmat)]
    for k in range(1, nmat):
        m = len(E[k-1])
        n = len(E[k])
        mat = np.zeros((m, n))
        for j in range(n):
            c = boundaryOperator(E[k][j])
            mat[:, j] = canonicalCoordinates(c, E[k-1])
        D[k-1] = mat
    return D


def projectedCanonicalCoordinates(c, K):
    ''' c is dict: {simplex: factor}
        K is list of simplices
    '''
    v = np.zeros(len(K))
    for ind in range(len(K)):
        if c.has_key(K[ind]):
            v[ind] = c[K[ind]]
    return v 

def maskedBoundaryOperatorMatrix(E, Emask):
    ''' Emask is the simplicial Chain groups you want to mask in
        It is the chain groups of the subsimplex
    '''
    nmat = len(E)-1
    D = [[] for i in range(nmat)]
    difflen = len(E) - len(Emask)
    for ind in range(difflen):
        Emask.append([])
    for k in range(1, nmat):
        m = len(E[k-1])
        n = len(E[k])
        mat = np.zeros((m, n))
        for j in range(n):
            if E[k][j] in Emask[k]:
                c = boundaryOperator(E[k][j])
                mat[:, j] = canonicalCoordinates(c, E[k-1])
        D[k-1] = mat
    return D

def laplacian(D, dim):
    
    Di = D[dim]
    Di1 = D[dim+1]
    return np.dot(Di.T, Di) + np.dot(Di1, Di1.T)

def expandBasis(mat, oldK, newK, oldKm1, newKm1):
    ''' oldK: source basis 1
        newK: source basis 2
        oldKm1: target basis 1
        newKm1: target basis 2
    '''
    basSource = sorted(union(oldK, newK))
    basTarget = sorted(union(oldKm1, newKm1))
    if mat == []:
        mat = np.zeros((len(basTarget), len(basSource)))
    else:    
        for ind, b in enumerate(basSource):
            if b not in oldK:
                mat = np.insert(mat, ind, 0, axis=1)
        for ind, b in enumerate(basTarget):
            if b not in oldKm1:
                mat = np.insert(mat, ind, 0, axis=0)
    return mat

def expandBases(D1, D2, E1, E2):
    newD1 = []
    newD2 = []
    minlen = min([len(D1), len(D2)])
    for ind in range(minlen-1):
        print(ind)
        print(D1[ind], D2[ind])
        newMat1 = expandBasis(D1[ind], E1[ind+1], E2[ind+1], E1[ind], E2[ind])
        newMat2 = expandBasis(D2[ind], E2[ind+1], E1[ind+1], E2[ind], E1[ind])
        newD1.append(newMat1)
        newD2.append(newMat2)
    return (newD1, newD2)

def densityMatrices(D, beta_list):

    rhos = []
    for ind in range(len(D)-3):
        L = laplacian(D, ind)
        M = spla.expm(beta_list[ind]*L)
        M = M / np.trace(M)
        rhos.append(M)
    return rhos

def KLdivergence(rho, sigma):
    r, w = np.linalg.eig(rho)
    s, w = np.linalg.eig(sigma)
    r = np.real(r)
    s = np.real(s)
    div = np.sum(np.multiply(r, (np.log(r) - np.log(s))/np.log(2.0)))
    return div

def JSdivergence(rho, sigma):
    
    M = (rho+sigma)/2.0
    return (KLdivergence(rho, M) + KLdivergence(sigma, M))/2.0

def JSdivergences(rho, sigma):
    
    assert (len(rho) == len(sigma))
    div = []
    for r, s in zip(rho, sigma):
        div.append(JSdivergence(r, s))
    return div

def spectralEntropies(rhos):

    ents = []
    for ind in range(len(rhos)):
        v, w = np.linalg.eig(rhos[ind])
        ve = np.log(v)
        ent = -np.dot(v.T, ve)
        ents.append(ent)
    return ents

def laplacians(D):

    l = len(D)
    laps = []
    for dim in range(1, len(D)-1):
        laps.append(laplacian(D, dim))
    return laps 

def stimSpaceGraph(E, D):
    ''' Takes a set of generators for the chain groups 
    and produces generators for the graph of the space
    '''
    E[0] = []
    Ec = [v for sl in E for v in sl]
    adj = np.zeros((len(Ec), len(Ec)))
    for k in range(1, len(E)-1):
        mat = np.array(D[k])
        lm1 = sum([len(E[i]) for i in range(k)])
        lm2 = lm1 + len(E[k])
        lm3 = sum([len(E[i]) for i in range(k+1)])
        lm4 = lm2 + len(E[k+1])
        adj[lm1:lm2, lm3:lm4] = np.abs(mat)
    adj = (adj + adj.T)
    return adj

def graphLaplacian(adj):
    
    D = np.diag(np.sum(adj, axis=0))
    L = D - adj
    return L

In [ ]:
adj = stimSpaceGraph(E, D)
L = graphLaplacian(adj)

In [ ]:
E = simplicialChainGroups(sorted([(1,2,3)]))
D = boundaryOperatorMatrix(E)

n = np.random.rand(15, 1200)
n = (n > 0.92).astype(int)
maxSimpList = sorted(sc.BinaryToMaxSimplex(n, rDup=True))
%time E = simplicialChainGroups(maxSimpList)
D = boundaryOperatorMatrix(E)
adj = stimSpaceGraph(E,D)
print(adj)
L = graphLaplacian(adj)
w, v = np.linalg.eig(L)
y, x, dontcare = plt.hist(np.real(w), 100)


plt.show()

In [ ]:
k=8
E[0] = []
lm1 = sum([len(E[i]) for i in range(k)])
lm2 = lm1 + len(E[k])
lm3 = sum([len(E[i]) for i in range(k+1)])
lm4 = lm2 + len(E[k+1])
print(lm1, lm2, lm3, lm4)

In [ ]:
len(E[5])

In [ ]:
from scipy.interpolate import interp1d
t = np.linspace(0, 30, 1000)
avg = np.zeros(len(t))
nreps = 60
for i in range(nreps):
    print(i)
    n = np.random.rand(15, 1200)
    n = (n > 0.90).astype(int)
    maxSimpList = sorted(sc.BinaryToMaxSimplex(n, rDup=True))
    E = simplicialChainGroups(maxSimpList)
    D = boundaryOperatorMatrix(E)
    adj = stimSpaceGraph(E,D)
    L = graphLaplacian(adj)
    w, v = np.linalg.eig(L)
    y, x, dontcare = plt.hist(np.real(w), 100)
    f = interp1d(x[:-1], y, kind='zero', bounds_error=False, fill_value=0)
    dat = f(t)
    avg = avg + dat
avg = avg / float(nreps)
plt.figure(figsize=(11,11))

plt.plot(t, avg)
plt.show()

In [ ]:
avg.shape

In [ ]:
E = simplicialChainGroups(sorted([(1,2), (3,4), (4,5), (1,4), (2,5)]), 1)
D = boundaryOperatorMatrix(E)
print(E)
print(D)

In [ ]:
n = np.random.rand(15, 1200)
n = (n > 0.97).astype(int)
maxSimpList = sorted(sc.BinaryToMaxSimplex(n, rDup=True))
%time E = simplicialChainGroups(maxSimpList)
D = boundaryOperatorMatrix(E)
L = laplacian(D, 2)
rhos = densityMatrices(D, np.ones(len(D)))
print(spectralEntropies(rhos))


In [ ]:
rhos

In [ ]:
D = boundaryOperatorMatrix(E)
w, v = np.linalg.eig(laplacian(D, 2))
y, x, dontcare = plt.hist(np.real(w), 100)
y

In [ ]:
c = boundaryOperator((1,2,3))
E = [(1,2), (1,3), (2,3), (4,5)]
canonicalCoordinates(c, E)

In [ ]:
set([1,2])

In [ ]:
set([]).union(*[[1,2], [2,3]])

In [ ]:
tuple([[1,2],[3,4]])

In [ ]:
np.outer([-4, -2, 3], [1,3,1])

# Expanding bases

In [ ]:
n = np.random.rand(15, 1200)
n = (n > 0.92).astype(int)
maxSimpList = sorted(sc.BinaryToMaxSimplex(n, rDup=True))
%time E = simplicialChainGroups(maxSimpList)
D = boundaryOperatorMatrix(E)

n2 = np.random.rand(15, 1200)
n2 = (n2 > 0.92).astype(int)
maxSimpList2 = sorted(sc.BinaryToMaxSimplex(n2, rDup=True))
%time E2 = simplicialChainGroups(maxSimpList2)
D2 = boundaryOperatorMatrix(E2)

In [ ]:
# E[i] is the basis for the i-1 chain group
# if mat = D[i] then mat: C_{i} to C_{i-1} 
# so new bases for D[i] is E[i+1], E[i]

newD_4 = expandBasis(D[4], E[5], E2[5], E[4], E2[4])
newD2_4 = expandBasis(D2[4], E2[5], E[5], E2[4], E[4])

In [ ]:
E[6]

In [ ]:
newD2_4.shape

In [3]:
ms1 = sorted([(1,2,3), (3,4), (4,5), (4,6), (5, 6)])
ms2 = sorted([(1,2,3), (4,5,6), (3,4)])
ms1 = sorted([(1,2,3)])
ms2 = sorted([(4,5,6,7)])
msTot = union(ms1, ms2)
E1 = simplicialChainGroups(ms1)
E2 = simplicialChainGroups(ms2)
Etot = simplicialChainGroups(msTot)
D1 = boundaryOperatorMatrix(E1)
D2 = boundaryOperatorMatrix(E2)
t = expandBases(D1, D2, E1, E2)

0
(array([[ 1.,  1.,  1.]]), array([[ 1.,  1.,  1.,  1.]]))
1
(array([[-1., -1.,  0.],
       [ 1.,  0., -1.],
       [ 0.,  1.,  1.]]), array([[-1., -1., -1.,  0.,  0.,  0.],
       [ 1.,  0.,  0., -1., -1.,  0.],
       [ 0.,  1.,  0.,  1.,  0., -1.],
       [ 0.,  0.,  1.,  0.,  1.,  1.]]))
2
(array([[ 1.],
       [-1.],
       [ 1.]]), array([[ 1.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.],
       [ 0., -1., -1.,  0.],
       [ 1.,  0.,  0.,  1.],
       [ 0.,  1.,  0., -1.],
       [ 0.,  0.,  1.,  1.]]))


/Users/brad/anaconda/envs/neuraltda/lib/python2.7/site-packages/ipykernel/__main__.py:119: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [4]:
D1exp = maskedBoundaryOperatorMatrix(Etot, E1)
D2exp = maskedBoundaryOperatorMatrix(Etot, E2)

In [7]:
rho1exp= densityMatrices(D1exp, np.ones(len(D1)))

In [9]:
n = np.random.rand(15, 1200)
n = (n > 0.92).astype(int)
maxSimpList = sorted(sc.BinaryToMaxSimplex(n, rDup=True))
%time E1 = simplicialChainGroups(maxSimpList)
D1 = boundaryOperatorMatrix(E1)

n2 = np.random.rand(15, 1200)
n2 = (n2 > 0.92).astype(int)
maxSimpList2 = sorted(sc.BinaryToMaxSimplex(n2, rDup=True))
%time E2 = simplicialChainGroups(maxSimpList2)
D2 = boundaryOperatorMatrix(E2)

Etot = simplexUnion(E1, E2)
D1exp = maskedBoundaryOperatorMatrix(Etot, E1)
D2exp = maskedBoundaryOperatorMatrix(Etot, E2)
rho1exp= densityMatrices(D1exp, np.ones(len(D1)))
rho2exp = densityMatrices(D2exp, np.ones(len(D2)))

CPU times: user 35.7 ms, sys: 72 µs, total: 35.7 ms
Wall time: 35.8 ms
CPU times: user 43.2 ms, sys: 259 µs, total: 43.5 ms
Wall time: 43.8 ms


[array([[ 0.06666667,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.06666667,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.06666667,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.06666667,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.06666667,
          0.        ,  0.        ,

[array([[ 0.06666667,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.06666667,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.06666667,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.06666667,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.06666667,
          0.        ,  0.        ,

In [19]:
JSdivergences(rho1exp, rho2exp)

[0.0, 3.8238864716669076, 0.19309699926754353]

In [ ]:
Dmax2 = boundaryOperatorMatrix(Etot)

In [ ]:
Dmax2